In [ ]:
import os
import pickle
import numpy as np
import PIL.Image
import torch
import matplotlib.pyplot as plt
import random
from scipy.interpolate import interp1d
import tensorflow as tf
from PIL.Image import Image as PilImage
import textwrap
import sys
import dnnlib.tflib as tflib

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
!git clone https://github.com/NVlabs/stylegan.git

In [ ]:
def get_save_interp_vectors(net_name, nz):
    if not os.path.exists('../interpol'):
        os.mkdir('../interpol')

    start = np.random.randn(1, nz)
    end = np.random.randn(1, nz)

    linfit = interp1d([1, 10], np.vstack([start, end]), axis=0)
    interp_vectors = [linfit(i) for i in range(1, 10 + 1)]

    np.savetxt(f'../interpol/{net_name}.csv', interp_vectors)

    return np.array(interp_vectors)

In [ ]:
def display_images(
    images: [PilImage], 
    columns=5, width=20, height=8, max_images=15, 
    label_wrap_length=50, label_font_size=8):

    if not images:
        print('No images to display.')
        return 

    if len(images) > max_images:
        print(f'Showing {max_images} images of {len(images)}:')
        images = images[0 : max_images]

    height = max(height, int(len(images) / columns) * height)
    plt.figure(figsize=(width, height))
    
    for i, image in enumerate(images):
        plt.subplot(int(len(images) / columns + 1), columns, i + 1)
        plt.imshow(image)

        if hasattr(image, 'filename'):
            title = image.filename
            if title.endswith('/'):
                title = title[0 : -1]
            title = os.path.basename(title)
            title = textwrap.wrap(title, label_wrap_length)
            title = '\n'.join(title)
            plt.title(title, fontsize=label_font_size); 

    fig_tosave = plt.gcf()
    if not os.path.exists('../interpol'):
        os.mkdir('../interpol')
    fig_tosave.savefig('../interpol/stylegan.png', dpi=600)

    plt.show()

In [ ]:
sys.path.insert(0, '/content/stylegan')

In [ ]:
def main():
    # Initialize TensorFlow.
    tflib.init_tf()

    # Load pre-trained network.
    with open('/content/karras2019stylegan-bedrooms-256x256 (2).pkl', 'rb') as f: 
        _G, _D, Gs = pickle.load(f)


    interp_vectors = get_save_interp_vectors('stylegan', Gs.input_shape[1])
    # Generate images.
    imgs = []
    for i, latent in enumerate(interp_vectors):
      latent = latent.reshape(1, -1)
      fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
      images = Gs.run(latent, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)

      # Append image.
      imgs.append(PIL.Image.fromarray(images[0], 'RGB'))
     
    
    display_images(imgs)

if __name__ == '__main__':
    main()